# Hashstack V0

This notebook processes Hashstack V0 events to get to the current state of all their loans. The data is then use to 
produce the main chart - liquidable debt vs. available supply.

In [1]:
import logging
import sys

sys.path.append('..')

import src.hashstack_v0
import src.main_chart
import src.swap_amm


logging.basicConfig(level=logging.INFO)

ModuleNotFoundError: No module named 'hashstack_v0'

In [ ]:
COLLATERAL_TOKEN = 'ETH'
DEBT_TOKEN = 'USDC'

## Prepare events

In [ ]:
events = src.hashstack_v0.get_events()

## Process events

In [ ]:
# Iterate over ordered events to obtain the final state of each user.
state = src.hashstack_v0.HashstackV0State(
    verbose_user='0x19629f075c1223d6f11a4428e9977a754dea1a8a789250b0879b2526a8840db',
)
for _, event in events.iterrows():
    state.process_event(event=event)

## Compute liquidable debt and available supply at various prices

In [ ]:
prices = src.swap_amm.Prices()
swap_amms = src.swap_amm.SwapAmm()
await swap_amms.init()
main_chart_data = src.main_chart.get_main_chart_data(
    state=state,
    prices=prices.prices,
    swap_amms=swap_amms,
    collateral_token=COLLATERAL_TOKEN,
    debt_token=DEBT_TOKEN,
)

## Plot the liquidable debt against the available supply

In [ ]:
src.main_chart.get_main_chart_figure(
    data=main_chart_data.astype(float),
    collateral_token=COLLATERAL_TOKEN,
    debt_token=DEBT_TOKEN,
)

In [ ]:
# # TODO: check selected user
# import src.settings
# USER = '0x38f554f6c5d0baa4088890bc00c9af2bfb0e80227e560443e3950805f57f54a'
# for token in src.settings.TOKEN_DECIMAL_FACTORS:
#     print(token, 'deposit', state.loan_entities[USER].deposit.values[token])
#     print(token, 'collateral', state.loan_entities[USER].collateral.values[token])
#     print(token, 'debt', state.loan_entities[USER].debt.values[token])

In [ ]:
# # TODO: negative balances?
# for attr in ['collateral', 'debt']:
#     for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT', 'wstETH']:
#         values = [(user, getattr(user_state, attr).values[token]) for user, user_state in state.loan_entities.items()]
#         min_value = min([x[1] for x in values])
#         min_value_user = [x[0] for x in values if min_value == x[1]][0]
#         print(
#             min_value_user,
#             attr,
#             token,
#             min_value,
#             round(prices.to_dollars(min_value, token), 2),
#         )

In [ ]:
# # TODO: check suspicious health factors
# for loan_entity_id, loan_entity in state.loan_entities.items():
#     health_factor = loan_entity.compute_health_factor(prices=prices.prices)
#     if 0 < health_factor < 0.7:
#         print(loan_entity_id, health_factor)